In [43]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

In [ ]:
df = pd.read_csv("Country_data.csv")
columns = df.columns[8:]
for column in columns:
    for i in range(0, len(df[column])):
        if df[column][i] != 'unknown':
            df.at[i, column] = df[column][i].replace(',','')
            df.at[i, column] = df[column][i].replace('$','')
            df.at[i, column] = df[column][i].replace('(','')
            df.at[i, column] = df[column][i].replace(')','')
        else:
            df.at[i, column] = None
df = df.drop(['Country Name.1'], axis = 1) 
df.to_csv('result.csv',index = False)

In [ ]:
df = pd.read_csv('result.csv')
columns = df.columns[7:]
for column in columns:
    mean_value = df[column].mean()
    for i in range(0, len(df[column])):
        if math.isnan(df[column][i]):
            df.at[i, column] = mean_value
df.to_csv('mean_country_data.csv',index = False)

In [81]:
df = pd.read_csv("mean_country_data.csv")
df= df[df.columns[7:]]
col = df.columns
x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(data = x_scaled, columns = col)
df_feature = df.drop('GDP - per capita (PPP)', 1)
df_feature

,Telephones - fixed lines,Telephones - mobile cellular,Internet users,Broadband - fixed subscriptions,GDP (purchasing power parity),GDP - real growth rate,Gross national saving,Industrial production growth rate,Labor force,Unemployment rate,Distribution of family income - Gini index,Taxes and other revenues,Budget surplus (+) or deficit (-),Public debt,Inflation rate (consumer prices),Central bank discount rate,Commercial bank prime lending rate,Stock of narrow money,Stock of broad money,Stock of domestic credit,Market value of publicly traded shares,Current account balance,Exports,Imports,Reserves of foreign exchange and gold,Debt - external,Stock of direct foreign investment - at home,Stock of direct foreign investment - abroad,Electricity - production,Electricity - consumption,Electricity - exports,Electricity - imports,Electricity - installed generating capacity,Electricity - from fossil fuels,Electricity - from nuclear fuels,Electricity - from hydroelectric plants,Electricity - from other renewable sources,Crude oil - production,Crude oil - exports,Crude oil - imports,Crude oil - proved reserves,Refined petroleum products - production,Refined petroleum products - consumption,Refined petroleum products - exports,Refined petroleum products - imports,Natural gas - production,Natural gas - consumption,Natural gas - exports,Natural gas - imports,Natural gas - proved reserves,Carbon dioxide emissions from consumption of energy,Area,Military expenditures,Population,Median age,Population growth rate,Birth rate,Death rate,Net migration rate,Maternal mortality rate,Infant mortality rate,Life expectancy at birth,Total fertility rate,Obesity - adult prevalence rate,Children under the age of 5 years underweight,Education expenditures,"Unemployment, youth ages 15-24",Airports,Railways,Roadways,Merchant marine
0,0.000613,0.016228,2.421502e-02,4.420670e-05,0.002737,0.390050,0.618585,0.280093,0.010505,0.307692,0.155000,0.086998,0.598223,0.017888,0.005421,0.210495,0.233521,0.000790,0.000823,0.000007,2.172773e-02,0.002258,3.148594e-04,0.003198,0.002221,0.000159,0.045884,4.687637e-02,0.000206,0.000993,0.000000e+00,0.060506,0.000383,0.450000,0.000000,0.520000,0.040000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.001734,0.000000,0.014651,0.000212,2.139135e-04,0.000000,0.000000e+00,0.001037,0.000777,0.038146,0.096288,0.025227,0.091384,0.523810,0.833333,0.655367,0.408016,0.554007,1.000000,0.000000,0.758621,0.057725,0.621827,0.262712,0.323251,0.003108,0.033830,0.005298,0.060533
1,0.001275,0.002368,2.752733e-03,7.754717e-04,0.001418,0.400995,0.524272,0.380787,0.001481,0.176010,0.145000,0.243786,0.727542,0.293867,0.002664,0.058333,0.118038,0.000492,0.000489,0.000295,2.172773e-02,0.002022,3.617269e-04,0.001710,0.001109,0.000531,0.001110,4.687637e-02,0.001213,0.000918,2.370023e-02,0.025124,0.001275,0.050000,0.000000,0.950000,0.000000,1.277139e-03,0.002355,0.000000,0.000557,0.000278,0.001433,0.000623,0.011418,0.000066,6.640177e-05,0.000000,0.000000e+00,0.000017,0.000386,0.001681,0.127610,0.002201,0.467363,0.326667,0.180108,0.299435,0.380267,0.011324,0.093545,0.710456,0.123412,0.332767,0.025381,0.254237,0.593573,0.000222,0.002276,0.000598,0.006518
2,0.016154,0.033828,2.365673e-02,8.365910e-03,0.024841,0.377114,0.828017,0.309028,0.014648,0.148631,0.302500,0.288719,0.652517,0.105196,0.005972,0.150000,0.114291,0.010121,0.010117,0.004029,2.172773e-02,0.049210,1.380321e-02,0.020532,0.030250,0.000350,0.005280,2.877337e-04,0.011370,0.010058,8.128329e-03,0.003534,0.011657,0.960000,0.000000,0.010000,0.020000,1.148513e-01,0.103038,0.000670,0.040357,0.030931,0.020271,0.110924,0.035516,0.120989,5.377801e-02,0.256327,0.000000e+00,0.094226,0.011645,0.139297,0.592807,0.030078,0.334204,0.453333,0.403226,0.152542,0.404933,0.095819,0.161833,0.672922,0.330309,0.429542,0.063452,0.298768,0.476371,0.011545,0.013503,0.015788,0.011600
3,0.000198,0.000049,1.079006e-04,9.036828e-05,0.000130,0.352239,0.595046,0.341423,0.000045,0.044329,0.395233,0.639579,0.679171,0.162692,0

In [99]:
no_feature = 10
pca = PCA(n_components=no_feature, svd_solver='randomized')
pca_feature = pca.fit_transform(df_feature)
col = ['pc'+str(i) for i in range(no_feature)]
pca_df = pd.DataFrame(data = pca_feature, columns = col)
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.27153854 0.15972877 0.11735296 0.05722073 0.04217174 0.03803508
 0.03221016 0.02512329 0.02311363 0.02253985]


In [100]:
pca_df['GDP - per capita (PPP)'] = df['GDP - per capita (PPP)']
train_set, test_set = train_test_split(pca_df,test_size=0.20,random_state=42)
Y_train = train_set['GDP - per capita (PPP)']
X_train = train_set.drop('GDP - per capita (PPP)', 1)
Y_test = test_set['GDP - per capita (PPP)']
X_test = test_set.drop('GDP - per capita (PPP)', 1)

In [101]:
lin_model= LinearRegression()
lin_model.fit(X_train,Y_train)
lin_Y_train_predict = lin_model.predict(X_train)
lin_train_mse = mean_squared_error(lin_Y_train_predict,Y_train)
lin_train_rmse = np.sqrt(lin_train_mse)
print("FOR TRAINING:")
print('Root MSE of the model:',lin_train_rmse, 'Mean of GDP:', Y_train.mean())
lin_train_r2 = r2_score(Y_train, lin_Y_train_predict)
print('R2 score is', lin_train_r2)

FOR TRAINING:
Root MSE of the model: 0.12590823980073756 Mean of GDP: 0.15382193123326843
R2 score is 0.5000597320172799


In [102]:
lin_Y_test_predict = lin_model.predict(X_test)
lin_test_mse = mean_squared_error(lin_Y_test_predict,Y_test)
lin_test_rmse = np.sqrt(lin_test_mse)
print("FOR TESTING:")
print('Root MSE of the model:',lin_test_rmse, 'Mean of GDP:', Y_test.mean())
lin_test_r2 = r2_score(Y_test, lin_Y_test_predict)
print('R2 score is', lin_test_r2)

FOR TESTING:
Root MSE of the model: 0.09778036779425373 Mean of GDP: 0.12158290998748586
R2 score is 0.39067842914699724
